In [ ]:
!pip3 install requests beautifulsoup4 pandas

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time

In [5]:
base_url = "http://ufcstats.com/statistics/fighters?char={}&page=all"
headers_conf = {"User-Agent": "Mozilla/5.0"}

fighter_list = []

for letter in range(65, 91):
  url = base_url.format(letter)
  resp = requests.get(url, headers=headers_conf)
  soup = BeautifulSoup(resp.content, 'html.parser')
  table = soup.find("table", class_="b-statistics__table")

  if table:
    links = table.find_all("a", href=True)
    for link in links:
      href = link["href"]
      if "/fighter-details/" in href:
        fighter_list.append(href)

print(f"Found {len(fighter_list)} Fighters.")

Found 336648 Fighters.


In [19]:
# Scrape profile data
fighter_data = []

def get_text(soup, selector):
  el = soup.select_one(selector)
  return el.get_text(strip=True) if el else ""

for link in fighter_list[:100]:
  resp = requests.get(link, headers=headers_conf)
  soup = BeautifulSoup(resp.content, "html.parser")

  name = get_text(soup, "span.b-content__title-highlight")
  height = get_text(soup, "li:contains('Height') .b-list__box-list-item")
  reach = get_text(soup, "li:contains('Reach') .b-list__box-list-item")
  stance = get_text(soup, "li:contains('Stance') .b-list__box-list-item")
  dob = get_text(soup, "li:contains('DOB') .b-list__box-list-item")
  record_raw = get_text(soup, "span.b-content__title-record")
  records = record_raw.replace("Record: ", "").split("-")

  wins, losses, draws = (records + [0,0,0])[:3]

  stats_table = soup.find("table", class_="b-list_table")
  stats = {}

  if stats_table:
    stats_rows = stats_table.find_all("tr")
    for row in stats_rows:
      cells = row.find_all("td")
      if len(cells) == 2:
        stat_name = cells[0].get_text(strip=True)
        stat_value = cells[1].get_text(strip=True)
        stats[stat_name] = stat_value

  fighter_data.append({
      "name": name,
      "height": height,
      "reach": reach,
      "stance": stance,
      "dob": dob,
      "wins": wins,
      "losses": losses,
      "draws": draws,
      **stats,
      "profile_url": link
  })

  time.sleep(2)

# convert to dataframe
df = pd.DataFrame(fighter_data)
# save to csv
df.to_csv("fighter_data.csv", index=False)

print("Scapped successfully and saved to fighter_data.csv.")

Scapped successfully and saved to fighter_data.csv.
